In [14]:
def ReduceCluster2(S, eps=10**-6, c=1, precision=2000):
    n = S[0].parent().codomain().dimension_relative() + 1
    cluster0 = [vector(list(pnt)) for pnt in S]
    m = len(cluster0)
    C = ComplexField(prec=precision)
    R = RealField(prec=precision)
    for v in cluster0:
        v.change_ring(C)
    cluster = [v*1/R(v.norm()) for v in cluster0]
    Tr = matrix.identity(n)
    count = 0
    total_count = 0
    if S[1][1].parent().is_exact():
        prec = precision
    else:
        prec = S[1][1].prec()*2
    while total_count < prec:
        total_count += 1
        cols = matrix(cluster).transpose()
        der1 = [R(col.hermitian_inner_product(col).real()) for col in cols]
        #der1 = [i.real() + i.imag() for i in der1] # fix small errors ???
        #der1 = [i.real().sign()*i.abs() for i in der1]
        der2 = []
        for i in range(n):
            temp_list = []
            for j in range(i):
                inner_prod = R(cols[i].hermitian_inner_product(cols[j]).real())
                #inner_prod = R(inner_prod.real()) + R(inner_prod.imag())
                #inner_prod = inner_prod.real().sign()*inner_prod.abs()
                temp_list.append(2*(inner_prod))
            der2.append(temp_list)
        average_der1 = sum(der1)/n
        der1 = [r - average_der1 for r in der1]
        absd = 0
        der2_prime = [] # unpack the list der2
        for lst in der2:
            der2_prime += lst
        for d in der1 + der2_prime:
            absd += d**2
        print('|der| =', absd)
        if absd < eps**2:
            break
        for i in range(len(der2)):
            der2[i].append(der1[i])
        
        #print('der2:', der2)
        der3 = []
        for lst in der2:
            new_lst = lst + [C(0)]*(n - len(lst))
            der3.append(new_lst)
            
        # make symmetric matrix
        B1 = matrix(der3)
        for i in range(n-1):
            for j in range(1 + i, n):
                B1[i, j] = B1[j, i]
        #print('B1:', B1)
        B = MatExp(-c*B1, eps=eps*1**-20).change_ring(R)
        if total_count == 1:
            print('B:', B)
        B *= B.determinant()**(-1/n)
        
        Tr *= B
        cluster1 = [B*pnt for pnt in cluster]
        val = sum(pnt.hermitian_inner_product(pnt).log() for pnt in cluster1)
        print('gain =', val)
        if val > 0:
            c /= 2
            print('new c =', c)
        else:
            count += 1
            if count == 10:
                count = 0
                Tr *= R(Tr.det())**(-1/n)
                cluster1 = [pt*Tr for pt in cluster0]
            cluster = [v*1/R(v.norm()) for v in cluster1]
    print(Tr)
    Tr = Tr*Tr.transpose()
    print(Tr)
    Tr = Tr.change_ring(RealField(prec=100))
    Tr = (Tr + Tr.transpose())/2
    U1 = Tr.LLL_gram().transpose()
    U1_inv = U1.inverse()
    return [v*U1_inv for v in cluster0], U1_inv, U1

In [2]:
def MatExp(mat, eps=1*10**-6):
    
    def mat_prod(mat):
        lst = []
        for tup in mat:
            for ele in tup:
                lst.append(ele)
        vec = vector(lst)
        return vec.hermitian_inner_product(vec)
        
    n = len(mat[0])
    result = matrix.identity(n)
    k = 1
    m = result
    while mat_prod(m) > eps^2:
        m *= 1/k * mat
        result += m
        k += 1
        #rint('m:', m)
        #print('k:', k)
    return result

In [3]:
C = ComplexField(prec = 2000)
P.<x,y,z> = ProjectiveSpace(QQ, 2)
X = P.subscheme(390908548757*x^4 - 1083699236751*x^3*y + 835578482044*x^3*z + 1126610184312*x^2*y^2 -
    1737329379412*x^2*y*z + 669777678687*x^2*z^2 - 520542386163*x*y^3 +
    1204081445939*x*y^2*z - 928398396271*x*y*z^2 + 238611653627*x*z^3 + 90192376558*y^4 -
    278168756247*y^3*z + 321720059816*y^2*z^2 - 165373310794*y*z^3 + 31877479532*z^4)
CR = P.coordinate_ring()
m = matrix(3, 3, [X.defining_polynomials()[0].derivative(CR.gens()[i]).derivative(CR.gens()[j]) for j in range(3) for i in range(3)])
Y = P.subscheme(m.det()) #hessian
R.<u,v> = PolynomialRing(QQ, order='lex')
subs = {x: u, y:v, z:1}
ideal = R.ideal([poly.subs(subs) for poly in X.defining_polynomials()] + [poly.subs(subs) for poly in Y.defining_polynomials()])
polys = ideal.groebner_basis()
Q = P.change_ring(C)
roots2 = [tup[0] for tup in polys[1].univariate_polynomial().change_ring(C).roots()]
roots1 = [-1*polys[0](0, root) for root in roots2]
pts = [Q((roots1[i], roots2[i], 1)) for i in range(len(roots1))]
len(pts)

24

In [13]:
Z = X.intersection(Y)
points = Z.rational_points(F=ComplexField(prec=1000))
ReduceCluster2(points)

|der| = 681.7391686993723703875093051530378643582775815749394356053090537166556411258102201184484145978538253227559833595106105534456887227811815436701781257136034206072284986505523361996872320650863777331092095389794393001951342346107968210811915603771617313673947043415313839786908109713802168066214387223198192480754128276800683317806706468100608731443225937334104119665679800966550711791771496652908801060885371033022368123146074928531574087849405883960459265565325271232289317422841215149313478103255036968130512530629180043203200741969674327953859506218644773874852596128315295073827678272081674029915611
gain = 556.257309475721172179534092590196318310235951871553922832926982827064788302315754918062281418269754235405456181487662488964486351884807092185881212039605798714975841580626823139414030386681216127784288527529809055228200334438736866862566315208685766732200723561787258628734148881822708421470397897314
new c = 1/2
|der| = 681.73916869937237038750930515303786435827758157493943560530

gain = -35.2473970133049963703933100425383192579501007585765344851510972155784735596597812063216332242884193777013227790889971749173046556601256015041227884330209419524963208885904028374013543932596863035300482093768350748377009827760232229392300174547738173702513925432622454870847406706962715376696062887818
|der| = 520.4195500067201893928799908066052658914577886425438761577426045970407032317956200660795449504603985803242804470586886651254661020898922544701539235484113908792368921518328826713665033970693681967145005061469835480701261484410690434997471710039834823679535384784357023701738044629521563992757069823504630366493610629038680463838066182069918019250082384092147765572479028618108061264507197990099704256245649804329626782895541180506250651334092469230613538982610343584035694102542798333415132117018415134217964890439183342121983618502088233049682675226431317998426811289803180411848622941285601968175546
gain = -29.85276557656034311430576859188188432149773976635081969553923304080311

|der| = 134.9960042538625924349388125890072130785959660787475951764665687129759211648794892325578652705276129320682171186948401743868792063706183089428541252314168896053386382337652689632975773176294387414803231364656748633303009682844424670450605773184858714675086572908251492682178827775728202981765032259781683675248707505708979452869299931760669909113985449844535997352297898862458284147886315738885458632546322297197239079928548850230678186206691797993512852408107602651184613530777754576165086808560084534665626606898325045343905512712860072446386177816374355079736462437991653861889996065896872871170817
gain = -12.0433507752139879012211071023465077885209409777151674367799915240082857030655049285093610108055370506720031503209467018914274936456098732531951513504898263950316717321873862903997619634824427183693876049605868160641857594796006334460182380022063571285092525978224961881714218406108298387675651385243
|der| = 135.0462904036054397185626184212210993046901358123870601185651860400457

|der| = 135.3502059582791147224694859438196564223714845574860838391950502827797858418450751513221749148686382670312342727076421765663010464969121603145449124944725151707902377277118763886021363417881421181989753406998938964672666623204353597489529468254754541603091948907595075000850276081535889895423218747286842512027812199233527669885717424804956806105369595223017241208634069095184948438884591625114166669232400413592177456522946770254372734453256756105278558248522560539719159568046714847455242162682413976606871671743962350345225699753484056432129074422979123269346129677859521595648363553782835245275779
gain = -12.0292439240873371044901191354352515236565875903269499792394244877188341236038165919921599959669676445884159972567431193414055871442193435786737550322464012074344681789514139600151384917281005468005647125688273483321511476241582462222790470889110354317649272751602661138410498625545398887893148163799
|der| = 135.0245880133759479002365125763371340395935588603826324820657940562895

(
[(-0.00121242564226390845339547008304905801195636146946536047875999845233364526986431351243904646910228556339944317548815849119327030885998787805016958649219359176637252758041304223513848186599737041906388511238456221797253421800283769765134673606447196684426823883247881849168628304234445719725216548038157 - 0.00449319276798033905540125743173719461615185491058679725193500067117969042722954338687679009085625551851039333986346076021636240013873596337858128659769400000707657115024147437369420721733927976170252529320391953362466440022283192660765009863949406680374201408748713196256990508909403400975777648759415*I, 0.00505562250229842208264709000219587436492804862386943507773418031659853230714705795887969055886572615177558430996103904923454929114793740434545007961634441534054444927893620419812257030167587779687440361652957939692980176704883303357414839091015812904936599112134912062320446002375917812221148011915187 + 0.00190318247736664557727279632392027761153573783051992703319650272210150

In [15]:
ReduceCluster2(pts, precision=500)

|der| = 681.739168699372370387509305153037864358277581574939435605309053716655641125810220118448414597853825322755983359510610553445688722781181543670178125713
B: [   66989.7930163162749341320490388871332067858773800649367741439787405232628537651527068509431894560921369739304032478480204199714863275771976907070527791 -3.20353218520440752116432717605763952174500986799500532550327216225192195254499752430987559884503020395073809421164761512251619002167482026704164165240e6  3.32360468983459765447305177530026936030179608709264896735763095155405434700815416839233903563316103349049891238863790904800969453080616869561101976221e6]
[-3.20353218520440752116432717605763952174500986799500532550327216225192195254499752430987559884503020395073809421164761512251619002167482026704164161209e6  1.69674860017821273518499740783572422648209495527863126055030283040709434286769748976303443451357240312460653706690576209058276014226721305864433320814e8 -1.76130533220252601689303209929987501957335892848326130309

gain = -12.0470071215937469529116345542261429851959654023978543906737383723297745019361401768307206151448906011801312935945780770122505696547342720548814428625
|der| = 107.390877269230713613607091083732578218666533713787528178655461253194129341003707728089036015204059452646816843301906145053127992610992624960173203940
gain = -11.9668583452783962920666084175878287056384825579655551721331426791683147695662520463422513974745704467547800907113965058357248005521645556780980452742
|der| = 108.252668551108694939891465454819356218018105478516570294326636304146958402810772336808181242078981560926657173688444665903080958990447882281114385153
gain = -11.9779217251469097644634344358076665337610162378101438843823842519987831236765162679650598825373615404892188250329631216041406714297773994296832961407
|der| = 109.228147024264138032275293228710299681425832544089094273325229582648197176270336870497568894544794040948827538911818237631652401525921345906582317077
gain = -11.95996950983343306354036704216

|der| = 134.879766181894811743611387032086069242754840202356942542387563590005055874621544362701330861831107259077972170917594323217821589153602542065292581339
gain = -12.0427411958640819560593111256195315725974636791734548422415065623251069817031949410343521287722699840621334802470384511017710236691757341759990931538
|der| = 134.942756604643021003799465906789264150285257869432802664792904169311149945924612827830050422661344571023521303594298181442345112419821161110233042431
gain = -11.9533190271164304696042063373400593599571650294225566996987608415015524981618791060791620910618280911990685813278871415339220938436599164271881379253
|der| = 134.996004253862592434938812589007213078595966078747595176466568712975921164879489232557865270527612932068217118694840174386879206370618308943171211714
gain = -12.0433507752139879012211071023465077885209409777151674367799915240082857030655049285093610108055370506720031503209467018914274936456098732532311647565
|der| = 135.0462904036054397185626184212

|der| = 15.7662582502336771267619088190115760835459667241668145513205194044165443257882504709385341341851629390195647716348171724072784282628363072895242616428
gain = -0.596564512121058676705061513601618462774202773579508312291716756820328504785128861468831710186989349140123764601565747965294646005873821224886377516943
|der| = 3.55147125813316548575309661576397131092872919152235459551061165274513034199406696241745876256031162820443044796724301491654306101421244903608983450461
gain = -0.119765489636787120023170485403532175412698886345949633916317994875513888856505503480209490906503955604151839652723151105012422645317277108888488865098
|der| = 0.781160080471494108446923588542831962427361500460800520457189394463109125428925070915653913775003491411327373013303484235463262075905814155941643277144
gain = -0.0254463339458217658739499130201914929919581336620668185860709696635872044488944064426046346143183876125816107953480592049400379585438175630986913019884
|der| = 0.1813696533601540959002231

(
[(0.001954237168973687017814627637058693668964048051231402634142131915184479271692616040239099958794527302123642745132265586841729704069703848465296781795707032956803635807085406691621153655905578652453541557493945971496647990767472513917892636403045544626074431689262387625648016755513520157473344768210885776412345244721697257683368629339394258259423916449296215450992209439154011945252139760681336224776198719064529339963656368992333095398219112810877009444137835713900811395903079828207327865924640999241344347028963927373439679040840049486969575212934799866858033554308485652811707343925627783671983784, 0.002013411342522574478637814226300920529092136235493759455066605660343409437519288310864112089944811466146447178865503310449849189185144555490039522145912291836795233290235237557177431992836716042141772029389389284518401472533506961804955889529764046485080650831249538140401591253080297078284775212985315899104182374107714150078218850306458268983698127752888578391693517241999976736838404

In [32]:
import_statements(vector)

# ** Warning **: several names for that object: free_module_element, vector
from sage.modules.free_module_element import free_module_element


In [41]:
R.<x> = QQ[]
K.<k> = NumberField(x^2 + 1)
P.<x,y,z> = ProjectiveSpace(K, 2)
pnts = [P(1,k,0), P(0,1,0), P(0,0,1), P(1,1,1)]
ReduceCluster2(pnts)

TypeError: unsupported operand parent(s) for *: 'Vector space of dimension 3 over Number Field in k with defining polynomial x^2 + 1' and 'Real Field with 2000 bits of precision'

In [19]:
t = RR(4)
t.nth_root(3)

1.58740105196820

In [21]:
t.inverse_of_unit()

0.250000000000000